In [1]:
import xmlhocr as xh
import webdataset as wds
import iautils
import io
from itertools import islice
import sys
import tempfile
import os

os.environ["RAY_DEDUP_LOGS"] = "0"

In [2]:
exn = Exception

def get_samples_full(url):
    ds = wds.WebDataset(url).decode()
    try:
        for sample in ds:
            key = sample["__key__"]
            print(f"key={key}", sample.keys())
            identifier = bytes.fromhex(key).decode("utf-8")

            try:
                pages = list(xh.convert_abbyy_to_serial(sample["abbyy.gz"]))
            except exn as e:
                print("failed to parse abbyy", repr(e))
                continue
            try:
                images = iautils.Jp2Zip(io.BytesIO(sample["zip"]))
            except exn as e:
                print("failed to load zip", repr(e))
                continue

            if set(images.pages.keys()) == set(range(len(pages))):
                print("zero-based page numbers in images")
            elif set(images.pages.keys()) == set(range(1, len(pages)+1)):
                print("one-based page numbers in images")
            else:
                print("page number mismatch", len(pages), len(images))
                continue

            page_offset = 0 if 0 in images else 1

            for i in range(len(pages)):
                pagekey = f"{identifier}/{i:05d}"
                print(pagekey, file=sys.stderr)
                image = images.decode(i+page_offset)
                sample = {
                    "__key__": pagekey,
                    "jpg": image,
                    "words.json": pages[i],
                }
                yield sample
    except Exception as e:
        print("exception", repr(e))
        raise e

In [3]:
exn = Exception

def get_samples(url):
    ds = wds.WebDataset(url).decode()
    try:
        for sample in ds:
            key = sample["__key__"]
            identifier = bytes.fromhex(key).decode("utf-8")
            print(identifier, key)

            try:
                pages = list(xh.convert_abbyy_to_serial(sample["abbyy.gz"]))
            except exn as e:
                print("failed to parse abbyy", repr(e))
                continue

            for i in range(len(pages)):
                pagekey = f"{key}/{i:05d}"
                # print(pagekey, file=sys.stderr)
                sample = {
                    "__key__": pagekey,
                    "lin.json": pages[i],
                }
                yield sample
    except Exception as e:
        print("exception", repr(e))
        raise e


In [4]:
if False:
    for sample in islice(get_samples("gs://ocro-iaa/books/books-000000.tar"), 30, 40):
        break
    sample

In [5]:
!gsutil ls gs://ocro-iaa/books/ | shardsum

gs://ocro-iaa/books/books-{000000..000703}.tar


In [6]:
def convert_books_to_lin(srcurl, dsturl):
    print("=== converting", srcurl, "-->", dsturl, file=sys.stderr)
    with tempfile.NamedTemporaryFile() as f:
        dst = wds.TarWriter(f.name)
        for sample in get_samples(srcurl):
            dst.write(sample)
        dst.close()
        f.flush()
        assert os.system(f"gsutil cp {f.name} {dsturl}") == 0


In [7]:
# convert_books_to_lin("gs://ocro-iaa/books/books-000001.tar", "gs://ocro-iaa/lin/lin-000001.tar")

In [8]:
import ray

if not ray.is_initialized():
    ray.init()

@ray.remote(num_cpus=4, memory=int(8e9))
def process_shard(shard):
    srcurl = f"gs://ocro-iaa/books/books-{shard:06d}.tar"
    dsturl = f"gs://ocro-iaa/lin/lin-{shard:06d}.tar"
    try:
        convert_books_to_lin(srcurl, dsturl)
    except Exception as e:
        return (shard, repr(e))
    return (shard, True)

# List of filenames to process
fnames = list(range(0, 704))

# Submit tasks to Ray in parallel
results = [process_shard.remote(fname) for fname in fnames]

# Wait for all tasks to complete and retrieve results
results = ray.get(results)

2023-05-11 06:55:52,189	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(process_shard pid=2077833) === converting gs://ocro-iaa/books/books-000003.tar --> gs://ocro-iaa/lin/lin-000003.tar
(process_shard pid=2077841) === converting gs://ocro-iaa/books/books-000001.tar --> gs://ocro-iaa/lin/lin-000001.tar
(process_shard pid=2077828) === converting gs://ocro-iaa/books/books-000000.tar --> gs://ocro-iaa/lin/lin-000000.tar
(process_shard pid=2077836) === converting gs://ocro-iaa/books/books-000002.tar --> gs://ocro-iaa/lin/lin-000002.tar


(process_shard pid=2077833) carletoncovof1986carl 6361726c65746f6e636f766f66313938366361726c


(process_shard pid=2077841) ERIC_ED018125 455249435f4544303138313235


(process_shard pid=2077836) mylittlewarexper00inspan 6d796c6974746c6577617265787065723030696e7370616e


...

(process_shard pid=2077833) curvesofcivileng00haynrich 6375727665736f66636976696c656e6730306861796e72696368


(process_shard pid=2077833) a588314000huxluoft 613538383331343030306875786c756f6674


(process_shard pid=2077833) watermanagementp00borcrich 77617465726d616e6167656d656e74703030626f726372696368


(process_shard pid=2077833) Copying file:///tmp/tmpgcwoca7k [Content-Type=application/octet-stream]...
(process_shard pid=2077833) / [0 files][    0.0 B/ 34.0 MiB]                                                


-
(process_shard pid=2077833) Operation completed over 1 objects/34.0 MiB.                                     
